In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit 
from teslakit.database import Database
from teslakit.estela import spatial_gradient, mask_from_poly, Predictor
#from teslakit.plotting.estela import Plot_ESTELA_Globe



## Database and Site parameters

In [3]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('GUAM')


In [4]:
# --------------------------------------
# load data and set parameters

ls_sea_poly = db.Load_ESTELA_coast()         # ESTELA coastline (land mask)
xds_est = db.Load_ESTELA_data()              # ESTELA D data
xds_WAVES = db.Load_ESTELA_waves()           # ESTELA gow waves point
xds_SLP_site = db.Load_ESTELA_SLP()          # ESTELA Sea Level Pressure

xds_wvs_pts = db.Load_WAVES_partitions()     # waves partitions data

_, xds_TCs_r1_params = db.Load_TCs_r1()      # TCs historical parameters inside 14º radius 

# KMA REGRESSION GUIDED parameters
kma_date_ini = '1979-01-22'
kma_date_end = '2011-01-22'
num_clusters = 36
kmarg_alpha = 0.36

# waves point coordinates
pnt_lon = 144.5
pnt_lat = 13.5

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (('time',), 144.5) to Variable.


## ESTELA Predictor - Sea Level Pressure and Gradient

In [ ]:
# --------------------------------------
# Plot ESTELA 

estela_D = xds_est.D_y1993to2012

# plot estela days-mask and wave point coordinates
Plot_ESTELA_Globe(pnt_lon, pnt_lat, estela_D)


In [ ]:
# --------------------------------------
# ESTELA waves data: calculate Fe, resample to daily

# calculate Fe
hs = xds_WAVES.hs
tm = xds_WAVES.t02
Fe = np.multiply(hs**2,tm)**(1.0/3)
xds_WAVES.update({
    'Fe':(('time',), Fe)
})

# select time window and calculate daily mean
xds_WAVES = xds_WAVES.sel(
    time = slice(kma_date_ini, kma_date_end)
).resample(time='1D').mean()

print(xds_WAVES)


In [ ]:
# --------------------------------------
# Calculate site SLP & SLP_gradient (daily)

# select time window and do data daily mean
xds_SLP_day = xds_SLP_site.sel(
    time = slice(kma_date_ini, kma_date_end)
).resample(time='1D').mean()

# calculate daily gradients
xds_SLP_day = spatial_gradient(xds_SLP_day, 'SLP')

# use coastline polygons to generate a land mask
xds_SLP_day = mask_from_poly(xds_SLP_day, ls_sea_poly, 'mask_land')
    

In [ ]:
# --------------------------------------
# Select site ESTELA and generate ESTELA land mask

# select ESTELA at site grid
xds_est_site = xds_est.sel(
    longitude = xds_SLP_site.longitude,
    latitude = xds_SLP_site.latitude,
)
estela_D = xds_est_site.D_y1993to2012

# use ESTELA to generate ESTELA mask
mask_est = np.zeros(estela_D.shape)
mask_est[np.where(estela_D<1000000000)]=1

xds_SLP_day.update({'mask_estela':(('latitude','longitude'), mask_est)})

print(xds_SLP_day)



## ESTELA Predictor - Principal Component Analysis

In [ ]:
# --------------------------------------
# Calculate ESTELA PCA

# Use our custom ESTELA Predictor 
pred = Predictor(db.paths.site.ESTELA.pred_slp)
pred.data = xds_SLP_day

# PCA (dynamic estela predictor)
pred.Calc_PCA_EstelaPred('SLP', estela_D)

print(pred.PCA)


In [ ]:
# plot PCA EOFs
n_EOFs = 3
pred.Plot_EOFs_EstelaPred(n_EOFs, show=True)



## ESTELA Predictor - KMeans Classification

In [ ]:
# Calculate KMA (regression guided with WAVES data)

# TODO: encontrar alpha optimo?
pred.Calc_KMA_regressionguided(
    num_clusters,
    xds_WAVES, ['hs','t02','Fe'],
    kmarg_alpha)

print(pred.KMA)


In [ ]:
# plot KMA clusters
pred.Plot_KMArg_clusters_datamean('SLP', show=True, mask_name='mask_estela')



## ESTELA Predictor - Add Historical TCs

In [ ]:
# --------------------------------------
# use historical storms-parameters inside r1 to modify predictor KMA results 

storm_dates = xds_TCs_r1_params.dmin_date.values[:]
storm_categs = xds_TCs_r1_params.category.values[:]

# add r1 storms to predictor KMA
pred.Mod_KMA_AddStorms(storm_dates, storm_categs)

print(pred.KMA)


In [ ]:
# --------------------------------------
# Calculate intradaily MU TAU hydrographs

l_xds_MUTAU = pred.Calc_MU_TAU_Hydrographs(xds_wvs_pts)

# store hydrographs MU TAU
db.Save_MU_TAU_hydrograms(l_xds_MUTAU)

# TODO: plot report hydrographs